In [5]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

In [6]:
df = pd.read_csv("insurance_data.csv")
df.head()

,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df[['age','affordibility']], df['bought_insurance'] , test_size = 0.2 , random_state = 25)


In [8]:
len(X_train)

22

In [9]:
# making age and affordability to the same scale so age is between 1 and 100 that is why dividing by 100

x_train_scaled = X_train.copy()
x_train_scaled['age'] = x_train_scaled['age']/100
x_train_scaled['age'].head()

x_test_scaled = X_test.copy()
x_test_scaled['age'] = x_test_scaled['age']/100
x_test_scaled['age'].head()

2     0.47
10    0.18
21    0.26
11    0.28
14    0.49
Name: age, dtype: float64

In [10]:
model = keras.Sequential([
    keras.layers.Dense(1 , input_shape = (2,) , activation = 'sigmoid' , kernel_initializer = 'ones' , 
                       bias_initializer = 'zeros')
])

c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [60]:
model.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)

model.fit(x_train_scaled , y_train , epochs = 1000)

Epoch 1/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 351ms/step - accuracy: 0.6364 - loss: 0.5942
Epoch 2/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.6364 - loss: 0.5941
Epoch 3/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.6364 - loss: 0.5940
Epoch 4/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.6364 - loss: 0.5939
Epoch 5/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.6364 - loss: 0.5938
Epoch 6/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.6364 - loss: 0.5937
Epoch 7/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.6364 - loss: 0.5937
Epoch 8/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.6364 - loss: 0.5936
Epoch 9/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.6364 - loss: 0.5935
Epoch 10/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.6364 - loss: 0.5934
Epoch 11/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.6364 - loss: 0.5933
Epoch 12/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy

In [61]:
coef , bias = model.get_weights()

In [62]:
coef , bias

(array([[2.6608963],
        [1.0637146]], dtype=float32),
 array([-1.6019835], dtype=float32))

In [63]:
# since the epochs are less we get less accuracy but for epochs 5000 we get more accuracy and less loss
model.evaluate(x_test_scaled , y_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - accuracy: 0.8333 - loss: 0.4794


[0.479362815618515, 0.8333333134651184]

In [14]:
# building a simple Neural Netwok

In [64]:
#sigmoid Function

def sigmoid(x):
    import math
    return 1/(1 + math.exp(-x))

sigmoid(18)

0.9999999847700205

In [65]:
# predicted Function

def predicted_function(age , affordability):
    weighted_sum = age*coef[0] + affordability*coef[1] + bias
    return sigmoid(weighted_sum)

predicted_function(.47, 1)

C:\Users\User\AppData\Local\Temp\ipykernel_7184\4087067945.py:5: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return 1/(1 + math.exp(-x))


0.6709207182888266

In [ ]:
#Now we start implementing gradient descent in plain python.
#  Again the goal is to come up with same w1, w2 and bias that keras model calculated. 
# We want to show how keras/tensorflow would have computed these values internally using gradient descent
#First write couple of helper routines such as sigmoid and log_loss

In [66]:
def sigmoid_array(X):
    return 1/(1 + np.exp(np.array(-X)))

sigmoid_array(np.array([12,0,1]))

array([0.99999386, 0.5       , 0.73105858])

In [67]:
# finding loss value using log_loss function
def log_loss(y_true , y_predicted):
    epsilon = 1e-15
    y_predicted_new = [max(i , epsilon) for i in y_predicted] 
    y_predicted_new = [min(i , epsilon) for i in y_predicted_new]
    y_predicted_new = np.array(y_predicted_new)
    return -np.mean(y_true*np.log(y_predicted_new)+(1-y_true)*np.log(1-y_predicted_new))

# y_predicted_new => Because np.log 0 is undefined in python so to avoid that taking 
# 0.999999 and 0.00000001(epsilon value(1e-15))

In [81]:
# Finally Implement the Gradient Decent Function
def gradient_decent(age , affor , y_true , epochs , loss_threshold):
    w1=w2=1
    bias=0
    learning_rate=0.5
    n=len(age)

    for i in range(epochs):
        weighted_sum = w1*age + w2*affor + bias
        y_predicted = sigmoid_array(weighted_sum)
        loss_value = log_loss(y_true , y_predicted)
        #Derivative Of W1 , w2 , bias
        w1_d = (1/n)*np.dot(np.transpose(age) , (y_predicted-y_true))
        w2_d = (1/n)*np.dot(np.transpose(affor) , (y_predicted-y_true))
        bias_d = np.mean(y_predicted-y_true) 

        w1 = w1 - (learning_rate*w1_d)
        w2 = w2 - (learning_rate*w2_d)
        bias = bias - (learning_rate*bias_d)

        print (f'Epoch:{i}, w1:{w1}, w2:{w2}, bias:{bias}, loss:{loss_value}')

        # if loss_value <= loss_threshold:
        #     break

    return w1 , w2 , bias

In [73]:
gradient_decent(x_train_scaled['age'],x_train_scaled['affordibility'],y_train,1000, 0.4631)

Epoch:0, w1:0.974907633470177, w2:0.948348125394529, bias:-0.11341867736368583, loss:17.26938819745534
Epoch:1, w1:0.9556229728273669, w2:0.9058873696677865, bias:-0.2122349122718517, loss:17.26938819745534
Epoch:2, w1:0.9416488476693794, w2:0.8719790823960313, bias:-0.2977578997796538, loss:17.26938819745534
Epoch:3, w1:0.9323916996249162, w2:0.8457541517722915, bias:-0.3715094724003511, loss:17.26938819745534
Epoch:4, w1:0.9272267472726993, w2:0.8262362885332687, bias:-0.43506643026891584, loss:17.26938819745534
Epoch:5, w1:0.9255469396815343, w2:0.8124402814952774, bias:-0.48994490058938817, loss:17.26938819745534
Epoch:6, w1:0.9267936114129968, w2:0.8034375029757677, bias:-0.5375299543522853, loss:17.26938819745534
Epoch:7, w1:0.93047170420295, w2:0.7983920007454487, bias:-0.5790424270894963, loss:17.26938819745534
Epoch:8, w1:0.9361540784567942, w2:0.7965748796787705, bias:-0.6155315088627655, loss:17.26938819745534
Epoch:9, w1:0.9434791243557357, w2:0.7973647616854131, bias:-0.64

(np.float64(8.270239516056172),
 np.float64(1.6332228396255535),
 np.float64(-4.425953608965705))

In [59]:
coef , bias

(array([[1.5983347 ],
        [0.51355225]], dtype=float32),
 array([-0.7191929], dtype=float32))

In [78]:
df.head()

,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


In [89]:
# Implementing neural network from scratch

class NeuralNetwork:
    def __init__(self):
        self.w1 = 1
        self.w2 = 1
        self.bias = 0

    def fit(self,x,y,epochs,loss_threshold):
        self.w1 , self.w2 , self.bias = gradient_decent(x['age'],x['affordibility'],
                                                             y,epochs,loss_threshold)

    
    def predict(self, X_test):
        weighted_sum = self.w1*X_test['age'] + self.w2*X_test['affordibility'] + self.bias
        return sigmoid_array(weighted_sum)
        

    
    
    

In [90]:
customModel = NeuralNetwork()
customModel.fit(x_train_scaled , y_train , epochs = 1000 , loss_threshold = 0.4631)

Epoch:0, w1:0.974907633470177, w2:0.948348125394529, bias:-0.11341867736368583, loss:17.26938819745534
Epoch:1, w1:0.9556229728273669, w2:0.9058873696677865, bias:-0.2122349122718517, loss:17.26938819745534
Epoch:2, w1:0.9416488476693794, w2:0.8719790823960313, bias:-0.2977578997796538, loss:17.26938819745534
Epoch:3, w1:0.9323916996249162, w2:0.8457541517722915, bias:-0.3715094724003511, loss:17.26938819745534
Epoch:4, w1:0.9272267472726993, w2:0.8262362885332687, bias:-0.43506643026891584, loss:17.26938819745534
Epoch:5, w1:0.9255469396815343, w2:0.8124402814952774, bias:-0.48994490058938817, loss:17.26938819745534
Epoch:6, w1:0.9267936114129968, w2:0.8034375029757677, bias:-0.5375299543522853, loss:17.26938819745534
Epoch:7, w1:0.93047170420295, w2:0.7983920007454487, bias:-0.5790424270894963, loss:17.26938819745534
Epoch:8, w1:0.9361540784567942, w2:0.7965748796787705, bias:-0.6155315088627655, loss:17.26938819745534
Epoch:9, w1:0.9434791243557357, w2:0.7973647616854131, bias:-0.64

In [95]:
model.predict(x_test_scaled)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step


array([[0.6709207 ],
       [0.48517743],
       [0.286969  ],
       [0.5515121 ],
       [0.6825619 ],
       [0.747416  ]], dtype=float32)

In [97]:
customModel.predict(x_test_scaled)

array([0.74918716, 0.21347787, 0.0931563 , 0.38294573, 0.77897215,
       0.90483284])